# LA's COVID-19 Reopening / Recovery Indicators 

The NY Times wrote about rushed reopenings being [a global trial-and-error experiment](https://www.nytimes.com/2020/05/07/world/europe/coronavirus-reopening-costs.html) and a further analysis showing how US states reopening [fail to meet the WH guidelines](https://www.nytimes.com/interactive/2020/05/07/us/coronavirus-states-reopen-criteria.html). Vox has written about [the importance of COVID testing indicators](https://www.vox.com/2020/5/8/21249880/coronavirus-testing-covid-reopening-economy-lockdowns-social-distancing) as a prerequisite for reopening. 

While we wait for LA's own reopening indicators, we can still see how LA stacks up against NY and Chicago's indicators. [NY State's Reopening Indicators](https://www.nytimes.com/2020/05/04/nyregion/coronavirus-reopen-cuomo-ny.html) were based heavily in part by the White House [Opening Up America Again gating criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf), and Chicago outlined even [tougher benchmarks than the IL statewide strategy](https://www.chicagotribune.com/coronavirus/ct-coronavirus-chicago-reopening-lightfoot-20200508-ztpnouwexrcvfdfcr2yccbc53a-story.html). In terms of outbreak severity, [LA and Chicago come right after NYC](https://www.nytimes.com/2020/05/09/us/coronavirus-chicago.html).

## [White House Gating Criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf)
### Symptoms
* Downward trajectory of influenza-like illnesses (ILI) reported within a 14-day period **and**
* Downward trajectory of COVID-like syndromic cases reported within a 14-day period

### Cases
* Downward trajectory of documented cases within a 14-day period **or**
* Downward trajectory of positive tests as a percent of total tests within a 14-day period (flat or increasing volume of tests) 

### Hospitals
* Treat all patients without crisis care **and**
* Robust testing program in place for at-risk healthcare workers, including emerging antibody testing

| NY | Chicago |
| --- | --- |
| At least 14 days decline in total net hospitalizations on a 3-day rolling average **or** <BR> Less than 15 net new total hospitalizations on a 3-day rolling average </BR> | At least 14 days decline in new cases, on average 
| At least 14 days decline in total net deaths on a 3-day rolling average **or** <BR> Less than 5 new deaths on a 3-day rolling average </BR> | At least 14 days decline in of case rate (per capita?) |
| At least 30% total hospital and ICU beds available  | At least 14 days decline in rate of new cases (per capita?)
| At least 90 days of PPE stockpiled  | Stable or declining rates of hospitalizations  
| Capacity to conduct 30 tests per 1,000 residents per month  | Stable or declining rates of ICU admissions 
| 30 working contact tracers per 100,000 residents | Stable or declining rates of ICU deaths
| Rural regions: Fewer than 2 new COVID patients admitted per 100,000 residents per day | Share of positive results are decreasing below 15% of those tested in the community 
|   | Fewer than 1,800 coronavirus patients in hospital beds 
|   | Fewer than 600 coronavirus patients in ICU beds
|   | Fewer than 450 coronavirus patients on ventilators
|   | Test 135,000 per month, equivalent to 4,500 people per day

In [1]:
import pandas as pd
import pytz

import utils
import meet_indicators

from datetime import date, datetime, timedelta
from IPython.display import Markdown


# Default parameters
county_state_name = "Los Angeles, CA"
state_name = "California"
msa_name = "Los Angeles-Long Beach-Anaheim, CA"
time_zone = "US/Pacific"

fulldate_format = "%-m/%-d/%y"
monthdate_format = "%-m/%-d"
start_date = pd.to_datetime("4/15/20").strftime(fulldate_format)
yesterday_date = utils.yesterday_date
today_date = utils.today_date
two_weeks_ago = utils.two_weeks_ago


# Daily testing upper and lower bound
test_lower_bound = 10_000
test_upper_bound = 16_667
positive_bound = 0.10
hospital_bound = 0.30

## Summary of Indicators

In [14]:
def summary_of_indicators():    
    la_county_case_indicator = meet_indicators.meet_case("county", county_state_name, start_date)
    la_county_death_indicator = meet_indicators.meet_death("county", county_state_name, start_date)
    lacity_case_indicator = meet_indicators.meet_lacity_case(start_date)
    
    test_indicator_color = meet_indicators.meet_daily_testing(yesterday_date, test_lower_bound, test_upper_bound)
    positive_indicator_color = meet_indicators.meet_positive_share(yesterday_date, positive_bound)
    
    acute_indicator = meet_indicators.meet_acute(yesterday_date)
    icu_indicator = meet_indicators.meet_icu(yesterday_date)
    ventilator_indicator = meet_indicators.meet_ventilator(yesterday_date)
    
    # Markdown output
    display(Markdown(f"### Indicators as of {yesterday_date}:"))
    display(Markdown("#### Cases")) 
    # LA County
    display(Markdown(f"LA County: declining cases: **{la_county_case_indicator}** out of 14 days"))
    display(Markdown(f"LA County: declining deaths: **{la_county_death_indicator}** out of 14 days"))
    # City of LA
    display(Markdown(f"City of LA: declining cases: **{lacity_case_indicator}** out of 14 days"))

    display(Markdown("#### Testing"))
    # Daily Testing
    if test_indicator_color == "blue":
        display(Markdown(f"Daily Testing: <strong><span style='color:{test_indicator_color}'>Exceeded</span></strong>"))    
    if test_indicator_color == "green":
        display(Markdown(f"Daily Testing: <strong><span style='color:{test_indicator_color}'>Met</span></strong>"))
    if test_indicator_color == "red":
        display(Markdown(f"Daily Testing: <strong><span style='color:{test_indicator_color}'>Not Met</span></strong>"))
    
    
    if positive_indicator_color == "green":
        display(Markdown(f"Share of Positive COVID-19 Results: <strong><span style='color:{test_indicator_color}'>Met</span></strong>"))          
    if positive_indicator_color == "red":
        display(Markdown(f"Share of Positive COVID-19 Results: <strong><span style='color:{test_indicator_color}'>Not Met</span></strong>"))   
        
    display(Markdown("#### Hospital Equipment"))  
    if acute_indicator >= hospital_bound:
        display(Markdown(f"City of LA % Acute Beds: <strong><span style='color:green'>{acute_indicator}</span></strong>"))          
    elif acute_indicator < hospital_bound:
        display(Markdown(f"City of LA % Acute Beds: <strong><span style='color:red'>{acute_indicator}</span></strong>"))          

    if icu_indicator >= hospital_bound:
        display(Markdown(f"City of LA % ICU Beds: <strong><span style='color:green'>{icu_indicator}</span></strong>"))          
    elif icu_indicator < hospital_bound:
        display(Markdown(f"City of LA % ICU Beds: <strong><span style='color:red'>{icu_indicator}</span></strong>")) 
    
    if ventilator_indicator >= hospital_bound:
        display(Markdown(f"City of LA % Ventilators: <strong><span style='color:green'>{ventilator_indicator}</span></strong>"))          
    elif ventilator_indicator < hospital_bound:
        display(Markdown(f"City of LA % Ventilators: <strong><span style='color:red'>{ventilator_indicator}</span></strong>")) 


In [10]:
def summary_of_indicators_statewide():

    state_case_indicator = meet_indicators.meet_case("state", state_name, start_date)
    state_death_indicator = meet_indicators.meet_death("state", state_name, start_date)
    msa_case_indicator = meet_indicators.meet_case("msa", msa_name, start_date)
    msa_death_indicator = meet_indicators.meet_death("msa", msa_name, start_date)
    
    # Markdown output
    display(Markdown(f"### Indicators as of {yesterday_date}:"))
    display(Markdown("#### Cases")) 
    
    # State
    display(Markdown(f"{state_name}: declining cases: **{state_case_indicator}** out of 14 days"))
    display(Markdown(f"{state_name}: declining deaths: **{state_death_indicator}** out of 14 days"))
    
    # MSA
    display(Markdown(f"{msa_name}: declining cases: **{msa_case_indicator}** out of 14 days"))
    display(Markdown(f"{msa_name}: declining deaths: **{msa_death_indicator}** out of 14 days"))

In [16]:
summary_of_indicators()
summary_of_indicators_statewide()

### Indicators as of 5/20/20:

#### Cases

LA County: declining cases: **5** out of 14 days

LA County: declining deaths: **8** out of 14 days

City of LA: declining cases: **7** out of 14 days

...

#### Cases

California: declining cases: **6** out of 14 days

California: declining deaths: **7** out of 14 days

Los Angeles-Long Beach-Anaheim, CA: declining cases: **4** out of 14 days

Los Angeles-Long Beach-Anaheim, CA: declining deaths: **8** out of 14 days

## Caseload Charts

In [5]:
la_county = utils.county_case_charts(county_state_name, start_date)
la_city = utils.lacity_case_charts("4/1/20") 
ca = utils.state_case_charts(state_name, start_date)
la_msa = utils.msa_case_charts("Los Angeles", start_date)

alt.HConcatChart(...)

alt.Chart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

## Testing Charts

In [6]:
daily_tests = utils.lacity_testing_charts(start_date, test_lower_bound, test_upper_bound)
positive_tests = utils.lacity_positive_test_charts(start_date)

alt.LayerChart(...)

alt.Chart(...)

alt.Chart(...)

## Hospital Capacity Charts

In [7]:
hospital = utils.lacity_hospital_charts(start_date)

alt.LayerChart(...)

alt.Chart(...)

In [8]:
"""To convert to HTML: 
jupyter nbconvert --to html --no-input --no-prompt my-notebook.ipynb
"""

'To convert to HTML: \njupyter nbconvert --to html --no-input --no-prompt my-notebook.ipynb\n'